In [87]:
#standard packages
import pandas as pd
import numpy as np
import sklearn.datasets

#preprocessing
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler


#results analysis
from sklearn.metrics import confusion_matrix, precision_score, classification_report

# models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

#pipeline
from sklearn.pipeline import Pipeline

#visualize
from sklearn.datasets import load_iris
from sklearn import tree

#dataframe loaded
df = pd.read_csv('data.csv')


In [88]:
#preprocessing and encoding columns

# numeric_features = ['duration',
#                     'credit_amount',
#                     'installment_commitment',
#                     'residence_since',
#                     'age',
#                     'existing_credits',
#                     'num_dependents']
# categorical_features = ['checking_status',
#                        'credit_history',
#                        'purpose',
#                        'savings_status',
#                       'employment',
#                       'personal_status',
#                       'other_parties',
#                        'property_magnitude',
#                        'other_payment_plans',
#                       'housing',
#                       'job',
#                       'own_telephone',
#                       'foreign_worker']


# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Need to find a way to use this to reduce dimensionality and improve speed:
#         ('reduce_dims', PCA(n_components=4))





preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="object")),
    ('cat', categorical_transformer, selector(dtype_include="object"))
])

# TESTING

In [89]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [90]:
#check for missing values
df.isnull().sum(axis = 0)

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [91]:
# Drop un-needed columns
df = df.drop(columns=['id','Unnamed: 32'])

In [92]:
df.isna().sum()

diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

In [93]:
df['diagnosis'] = LabelEncoder().fit_transform(df['diagnosis'])
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [98]:
df.corr(method='pearson')

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
diagnosis,1.000000,0.730029,0.415185,0.742636,0.708984,0.358560,0.596534,0.696360,0.776614,0.330499,...,0.776454,0.456903,0.782914,0.733825,0.421465,0.590998,0.659610,0.793566,0.416294,0.323872
radius_mean,0.730029,1.000000,0.323782,0.997855,0.987357,0.170581,0.506124,0.676764,0.822529,0.147741,...,0.969539,0.297008,0.965137,0.941082,0.119616,0.413463,0.526911,0.744214,0.163953,0.007066
texture_mean,0.415185,0.323782,1.000000,0.329533,0.321086,-0.023389,0.236702,0.302418,0.293464,0.071401,...,0.352573,0.912045,0.358040,0.343546,0.077503,0.277830,0.301025,0.295316,0.105008,0.119205
perimeter_mean,0.742636,0.997855,0.329533,1.000000,0.986507,0.207278,0.556936,0.716136,0.850977,0.183027,...,0.969476,0.303038,0.970387,0.941550,0.150549,0.455774,0.563879,0.771241,0.189115,0.051019
area_mean,0.708984,0.987357,0.321086,0.986507,1.000000,0.177028,0.498502,0.685983,0.823269,0.151293,...,0.962746,0.287489,0.959120,0.959213,0.123523,0.390410,0.512606,0.722017,0.143570,0.003738
smoothness_mean,0.358560,0.170581,-0.023389,0.207278,0.177028,1.000000,0.659123,0.521984,0.553695,0.557775,...,0.213120,0.036072,0.238853,0.206718,0.805324,0.472468,0.434926,0.503053,0.394309,0.499316
compactness_mean,0.596534,0.506124,0.236702,0.556936,0.498502,0.659123,1.000000,0.883121,0.831135,0.602641,...,0.535315,0.248133,0.590210,0.509604,0.565541,0.865809,0.816275,0.815573,0.510223,0.687382
concavity_mean,0.696360,0.676764,0.302418,0.716136,0.685983,0.521984,0.883121,1.000000,0.921391,0.500667,...,0.688236,0.299879,0.729565,0.675987,0.448822,0.754968,0.884103,0.861323,0.409464,0.514930
concave points_mean,0.776614,0.822529,0.293464,0.850977,0.823269,0.553695,0.831135,0.921391,1.000000,0.462497,...,0.830318,0.292752,0.855923,0.809630,0.452753,0.667454,0.752399,0.910155,0.375744,0.368661
symmetry_mean,0.330499,0.147741,0.071401,0.183027,0.151293,0.557775,0.602641,0.500667,0.462497,1.000000,...,0.185728,0.090651,0.219169,0.177193,0.426675,0.473200,0.433721,0.430297,0.699826,0.438413


In [99]:
y = df.iloc[:, 0:1]

X = df.drop('diagnosis',axis='columns')
X = X.iloc[:,:]
print(X)
X.info

# target_feature = ['diagnosis']

subset_feature = [
                'radius_mean',
#                 'texture_mean',
                'perimeter_mean',
                'area_mean',
#                 'smoothness_mean',
#                 'compactness_mean',
#                 'concavity_mean',
                'concave points_mean',
#                 'symmetry_mean',
#                 'fractal_dimension_mean',
#                 'radius_se',
#                 'texture_se',
#                 'perimeter_se',
#                 'area_se',
#                 'smoothness_se',
#                 'compactness_se',
#                 'concavity_se',
#                 'concave points_se',
#                 'symmetry_se',
#                 'fractal_dimension_se',
                'radius_worst',
#                 'texture_worst',
                'perimeter_worst',
                'area_worst',
#                 'smoothness_worst',
#                 'compactness_worst',
#                 'concavity_worst',
                'concave points_worst',
#                 'symmetry_worst',
#                 'fractal_dimension_worst',
                 ]

X = X[subset_feature]
X.info()

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0          17.99         10.38          122.80     1001.0          0.11840   
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     compactness_mean  concavity_mean  concave points_mean  sym

In [100]:
y.info

<bound method DataFrame.info of      diagnosis
0            1
1            1
2            1
3            1
4            1
..         ...
564          1
565          1
566          1
567          1
568          0

[569 rows x 1 columns]>

In [114]:
#Pipeline

# MODEL TYPES

# KNeighborsClassifier(n_neighbors=K)

# LogisticRegression(solver=lbfgs, C=C)
#     solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’

# SVC(kernel='linear', C=C)
#     kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’)
#     degreeint, default=3 (for poly only)

# RandomForestClassifier(n_estimators=100, criterion='gini')

# DecisionTreeClassifier(criterion='gini', splitter='best')



model_choice = KNeighborsClassifier(n_neighbors=10, p=2)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('classifier', model_choice)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

cross_val_score(clf, X, y, scoring='accuracy').mean()

conf = confusion_matrix(y_test, y_pred)
print(conf)

print("Current model score: %.5f" % clf.score(X_test, y_test))


param_grid = dict(classifier__n_neighbors=[1,2,3,4,5,6,7,8,9,10],
                  classifier__p=[1,2])
# For future use - try using with PCA above
#     reduce_dims__n_components=[4,6,8]


grid = GridSearchCV(clf, param_grid=param_grid, cv=3, n_jobs=1, verbose=10, scoring= 'accuracy', return_train_score=True)
grid.fit(X, y)
print(grid.best_score_)
grid.cv_results_

#KNN - neighbours = 10, p = 2 was best choice, 94.737% accuracy

C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

[[78  4]
 [ 2 30]]
Current model score: 0.94737
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] classifier__n_neighbors=1, classifier__p=1 ......................
[CV]  classifier__n_neighbors=1, classifier__p=1, score=(train=1.000, test=0.926), total=   0.0s
[CV] classifier__n_neighbors=1, classifier__p=1 ......................


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s


[CV]  classifier__n_neighbors=1, classifier__p=1, score=(train=1.000, test=0.937), total=   0.1s
[CV] classifier__n_neighbors=1, classifier__p=1 ......................
[CV]  classifier__n_neighbors=1, classifier__p=1, score=(train=1.000, test=0.931), total=   0.1s
[CV] classifier__n_neighbors=1, classifier__p=2 ......................


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s


...

[CV]  classifier__n_neighbors=10, classifier__p=2, score=(train=0.953, test=0.926), total=   0.1s
[CV] classifier__n_neighbors=10, classifier__p=2 .....................
[CV]  classifier__n_neighbors=10, classifier__p=2, score=(train=0.960, test=0.958), total=   0.1s
[CV] classifier__n_neighbors=10, classifier__p=2 .....................
[CV]  classifier__n_neighbors=10, classifier__p=2, score=(train=0.958, test=0.974), total=   0.0s


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    7.3s finished


0.9525851666202544


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


{'mean_fit_time': array([0.03266199, 0.03698818, 0.04533625, 0.04292814, 0.0388337 ,
        0.04624891, 0.03840224, 0.03540373, 0.03519456, 0.0516746 ,
        0.03635478, 0.0375553 , 0.04536676, 0.04342151, 0.04162463,
        0.0354619 , 0.03338718, 0.03906385, 0.04141227, 0.03923273]),
 'std_fit_time': array([0.00423293, 0.00448922, 0.01559026, 0.00755782, 0.00995831,
        0.0096749 , 0.00684488, 0.00769272, 0.00732268, 0.0122772 ,
        0.00464965, 0.00553274, 0.01231867, 0.0011228 , 0.00827766,
        0.00365236, 0.00369739, 0.00869025, 0.00415095, 0.00638065]),
 'mean_score_time': array([0.0179371 , 0.02582932, 0.04984665, 0.030466  , 0.03287148,
        0.03124237, 0.02871943, 0.03063273, 0.02904288, 0.03801282,
        0.02314647, 0.02858663, 0.02723169, 0.02169506, 0.03315409,
        0.02849833, 0.02893877, 0.03180202, 0.03672353, 0.02328793]),
 'std_score_time': array([0.00824444, 0.00322035, 0.03248697, 0.00347508, 0.0091304 ,
        0.00506096, 0.00472366, 0.012112

In [116]:

model_choice = RandomForestClassifier(n_estimators=400, criterion='entropy', random_state=0)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('classifier', model_choice)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

cross_val_score(clf, X, y, scoring='accuracy').mean()

conf = confusion_matrix(y_test, y_pred)
print(conf)

print("Current model score: %.5f" % clf.score(X_test, y_test))


param_grid = dict(classifier__n_estimators=[1,10,100,200,400],
                  classifier__criterion=['gini','entropy'])
# For future use - try using with PCA above
#     reduce_dims__n_components=[4,6,8]


grid = GridSearchCV(clf, param_grid=param_grid, cv=3, n_jobs=1, verbose=10, scoring= 'accuracy', return_train_score=True)
grid.fit(X, y)
print(grid.best_score_)
grid.cv_results_

#Random Forest Classifier - estimators = 400, criterion = entropy was best choice, 94.737% accuracy

C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

[[77  3]
 [ 3 31]]
Current model score: 0.94737
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] classifier__criterion=gini, classifier__n_estimators=1 ..........
[CV]  classifier__criterion=gini, classifier__n_estimators=1, score=(train=0.974, test=0.921), total=   0.0s
[CV] classifier__criterion=gini, classifier__n_estimators=1 ..........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,),

[CV]  classifier__criterion=gini, classifier__n_estimators=1, score=(train=0.974, test=0.963), total=   0.0s
[CV] classifier__criterion=gini, classifier__n_estimators=1 ..........
[CV]  classifier__criterion=gini, classifier__n_estimators=1, score=(train=0.961, test=0.952), total=   0.1s
[CV] classifier__criterion=gini, classifier__n_estimators=10 .........
[CV]  classifier__criterion=gini, classifier__n_estimators=10, score=(train=0.997, test=0.916), total=   0.1s
[CV] classifier__criterion=gini, classifier__n_estimators=10 .........


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


...

C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


[CV]  classifier__criterion=entropy, classifier__n_estimators=400, score=(train=1.000, test=0.952), total=   2.0s


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   22.7s finished
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


0.9507936507936506


{'mean_fit_time': array([0.03944596, 0.06519723, 0.46308414, 0.84884906, 1.82858284,
        0.04988734, 0.07856886, 0.49982627, 0.88636605, 1.74294051]),
 'std_fit_time': array([0.00515283, 0.01217527, 0.03325036, 0.04934429, 0.08118047,
        0.01324382, 0.0010343 , 0.03904322, 0.0863243 , 0.0894121 ]),
 'mean_score_time': array([0.00525649, 0.01432093, 0.03364746, 0.05401476, 0.12781096,
        0.01161329, 0.01186736, 0.05042076, 0.07106614, 0.11651246]),
 'std_score_time': array([0.00372077, 0.00172723, 0.0133927 , 0.00387022, 0.0310485 ,
        0.0024662 , 0.00394727, 0.01428845, 0.02124772, 0.01384798]),
 'param_classifier__criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'entropy',
                    'entropy', 'entropy', 'entropy', 'entropy'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_classifier__n_estimators': masked_array(

In [117]:

model_choice = SVC(kernel='linear', C=10, random_state=0)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('classifier', model_choice)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

cross_val_score(clf, X, y, scoring='accuracy').mean()

conf = confusion_matrix(y_test, y_pred)
print(conf)

print("Current model score: %.5f" % clf.score(X_test, y_test))


param_grid = dict(classifier__C=[0.1,0.2,0.3,0.4,0.5,1,10,20,50,100],
                  classifier__kernel=['linear','poly','rbf'])
# For future use - try using with PCA above
#     reduce_dims__n_components=[4,6,8]


grid = GridSearchCV(clf, param_grid=param_grid, cv=3, n_jobs=1, verbose=10, scoring= 'accuracy', return_train_score=True)
grid.fit(X, y)
print(grid.best_score_)
grid.cv_results_

#SVC - kernal = linear, C = 50 was best choice, 97.368% accuracy

C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

[[62  1]
 [ 2 49]]
Current model score: 0.97368
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] classifier__C=0.1, classifier__kernel=linear ....................
[CV]  classifier__C=0.1, classifier__kernel=linear, score=(train=0.947, test=0.932), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=linear ....................
[CV]  classifier__C=0.1, classifier__kernel=linear, score=(train=0.942, test=0.953), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=linear ....................
[CV]  classifier__C=0.1, classifier__kernel=linear, score=(train=0.950, test=0.947), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=poly ......................


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
C:\U

[CV]  classifier__C=0.1, classifier__kernel=poly, score=(train=0.852, test=0.789), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=poly ......................
[CV]  classifier__C=0.1, classifier__kernel=poly, score=(train=0.828, test=0.821), total=   0.1s
[CV] classifier__C=0.1, classifier__kernel=poly ......................
[CV]  classifier__C=0.1, classifier__kernel=poly, score=(train=0.832, test=0.905), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=rbf .......................
[CV]  classifier__C=0.1, classifier__kernel=rbf, score=(train=0.942, test=0.905), total=   0.0s
[CV] classifier__C=0.1, classifier__kernel=rbf .......................

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s remaining:    0.0s
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

[CV]  classifier__C=100, classifier__kernel=poly, score=(train=0.939, test=0.916), total=   0.1s
[CV] classifier__C=100, classifier__kernel=poly ......................
[CV]  classifier__C=100, classifier__kernel=poly, score=(train=0.942, test=0.963), total=   0.1s
[CV] classifier__C=100, classifier__kernel=rbf .......................
[CV]  classifier__C=100, classifier__kernel=rbf, score=(train=0.979, test=0.937), total=   0.0s
[CV] classifier__C=100, classifier__kernel=rbf .......................


C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV]  classifier__C=100, classifier__kernel=rbf, score=(train=0.968, test=0.963), total=   0.0s
[CV] classifier__C=100, classifier__kernel=rbf .......................
[CV]  classifier__C=100, classifier__kernel=rbf, score=(train=0.974, test=0.968), total=   0.0s
0.9613385315139701


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    6.1s finished
C:\Users\iansc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


{'mean_fit_time': array([0.02401121, 0.03715086, 0.04185224, 0.03800782, 0.04827483,
        0.05872989, 0.04624724, 0.03224969, 0.03257593, 0.04522824,
        0.03633451, 0.04238168, 0.02999274, 0.03175728, 0.03619687,
        0.03523668, 0.03581834, 0.02974772, 0.02897739, 0.03954148,
        0.04411713, 0.0330886 , 0.03312667, 0.05106378, 0.03084453,
        0.0440228 , 0.03719489, 0.05137197, 0.0525016 , 0.02963217]),
 'std_fit_time': array([0.00086241, 0.00507716, 0.00715717, 0.00794762, 0.01459461,
        0.00578838, 0.01061432, 0.00553336, 0.00282078, 0.01423158,
        0.0030107 , 0.00763734, 0.00329779, 0.00195275, 0.00686187,
        0.00325776, 0.00053545, 0.00277444, 0.00153773, 0.0089978 ,
        0.0083386 , 0.00206439, 0.0036544 , 0.01695519, 0.00406983,
        0.01402386, 0.00640103, 0.00588408, 0.00861358, 0.00033158]),
 'mean_score_time': array([0.00273808, 0.00872334, 0.01213686, 0.01412574, 0.01484164,
        0.01405096, 0.01451484, 0.00667938, 0.01153898, 0.01